# Managing scraped data (11th December 2021)

This notebook manages the tribunal decision's data scraped in 0_dataScraping.ipynb

In particular, the notebook:

1. Stores the text of each tribunal decision in the corresponding object of jsonDataFinal (list of dict) with an uuid.

2. Converts the 35305 downloaded word documents to text (from .doc/.docx to .txt)

3. Provides some descriptive statistics on the downloaded files.

The resulting data set (a list of dictionaries) is serialised as a json object (jsonDataFinal.json).

This notebook should run in the tfm environment, which can be created with the environment.yml file.

In [1]:
from os import listdir
from os.path import isfile, join, getsize
import numpy as np
import time
import re
import json
import pickle
import pandas as pd
import whois
import sys
import datetime
from tqdm import tqdm
import textract
import wget
import uuid

import sys
IN_COLAB = 'google.colab' in sys.modules


# What environment am I using?
print(f'Current environment: {sys.executable}')

# Change the current working directory
os.chdir('/Users/albertamurgopacheco/Documents/GitHub/TFM')
# What's my working directory?
print(f'Current working directory: {os.getcwd()}')


Current environment: /Users/albertamurgopacheco/anaconda3/envs/tfm/bin/python
Current working directory: /Users/albertamurgopacheco/Documents/GitHub/TFM


In [6]:
# Define working directories in colab and local execution

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    docs_path = '/content/gdrive/MyDrive/TFM/data/raw'
    input_path = '/content/gdrive/MyDrive/TFM'
    output_path = '/content/gdrive/MyDrive/TFM/output'

else:
    docs_path = './data/raw'
    input_path = '.'
    output_path = './output'

# 1. Storing tribunal decisions in jsonDataFinal
A string with the tribunal decision's text is added to each dictionary in the jsonData list of dictionaries. The resulting collection is saved as jsonDataFinal.

In [12]:
# Path to jsonData file
jsonData_path = os.path.join(os.getcwd(), 'data/jsonData.json')

# Open jsonData file as data
with open(jsonData_path) as json_file:
    data = json.load(json_file)

In [13]:
# Loop over dictionaries and upload the string of the judicial decision
for d in tqdm(data):
    # Obtain the url to the file with the judicial decision
    docLink = d.get('Document')
    # Temp folder to store word file
    docs_temp = './data/temp/'
    # List of corrupted urls
    corruptFiles = ['HU077022015', 'HU029682017']
    
    if d.get('File') not in corruptFiles:

        try:
            # Download file to temp folder
            wget.download(url = docLink, out = docs_temp)
            # Delete DS_Store files in folder
            !find . -name '.DS_Store' -type f -delete
            # Get name of downloaded file 
            filename = os.listdir(docs_temp)
            filename = filename[0]
            # Extract text from the file as a string
            content = textract.process(os.path.join(docs_temp, filename))
            # Add content to dict key 'String':
            d.update({'String': content})
            # Delete the file
            os.remove(docs_temp + filename)
        
        # Handling exceptions 
        except Exception as err:
            print("Could not download file {}".format(docLink))
            print(err)
            downloaded = "No"
            pass
    else:
        continue

  0%|          | 49/35308 [00:46<7:08:56,  1.37it/s]

Could not download file https://moj-tribunals-documents-prod.s3.amazonaws.com/decision/doc_file/73729/DC000912019___DC001272019.doc
HTTP Error 403: Forbidden


 92%|█████████▏| 32492/35308 [7:36:44<25:52,  1.81it/s]

Could not download file https://moj-tribunals-documents-prod.s3.amazonaws.com/decision/doc_file/40081/IA083642010___IA083692010___IA083752010.DOC
HTTP Error 403: Forbidden


100%|██████████| 35308/35308 [8:13:21<00:00,  1.19it/s]


In [14]:
# Decode in utf-8 the data saved in bytes in 'String'
for d in tqdm(data):
    string = d.get('String')
    # The stored value is in bytes and should be decoded
    if not string:
        continue
    else: 
        string = string.decode('utf-8')
        # Add dictionary key 'String' with value string
        d.update({'String': string})
        #print(d)
        
# Save data as a json file jsonDataFinal in data directory
with open('./data/jsonDataFinal.json', 'w') as fout:
    json.dump(data, fout)



100%|██████████| 35308/35308 [00:00<00:00, 70497.30it/s]


In [15]:
# Save data as a json file jsonData in data directory
with open('./data/jsonDataFinalBackup.json', 'w') as fout:
    json.dump(data, fout)

In [16]:
# Save as a pickle
with open('./data/pickleDataFinal.pkl', 'wb') as f:
    pickle.dump(data, f, protocol = pickle.HIGHEST_PROTOCOL)

In [17]:
# Open pickle file 
with open('./data/pickleDataFinal.pkl', 'rb') as f:
    data = pickle.load(f)

Pop entries with corrupt word files. No text is available for these entries. ['HU077022015', 'HU029682017']

In [18]:

# Open jsonData file as data
with open('./data/jsonDataFinal.json') as json_file:
    data = json.load(json_file)

# List of corrupted urls
corruptFiles = ['HU077022015', 'HU029682017', 'IA083642010___IA083692010___IA083752010']

# Loop over dictionaries and remove the corrupt files entries
for d in tqdm(data):
    if d['File'] in corruptFiles:
        # Remove elements
        idx = data.index(d)
        del data[idx]
        print(f'Dict indexed {idx} has been removed from list.')
    
# Save data as a json file jsonDataFinal in data directory
with open('./data/jsonDataFinal.json', 'w') as fout:
    json.dump(data, fout)

100%|█████████▉| 35305/35308 [00:00<00:00, 2498817.12it/s]


Dict indexed 10481 has been removed from list.
Dict indexed 12643 has been removed from list.
Dict indexed 32489 has been removed from list.


Add a unique ID to each dict in the jsonDataFinal using uuid.

In [19]:
# Open jsonData file as data
with open('./data/jsonDataFinal.json') as json_file:
    data = json.load(json_file)

# Loop over dictionaries and remove the corrupt files entries
for d in tqdm(data):
    # Generate random uuid
    id = str(uuid.uuid4())
    # Add content to dict key 'String':
    d.update({'ID': id})

# Save data as a json file jsonDataFinal in data directory
with open('./data/jsonDataFinal.json', 'w') as fout:
    json.dump(data, fout)


100%|██████████| 35305/35305 [00:00<00:00, 233505.22it/s]


# 2. Converting word documents to text (from .doc/.docx to .txt)

The 35258 downloaded word documents (.doc/.docx) are converted to text (.txt) format.

In [12]:
# Delete DS_Store files in raw data folder
!find . -name '.DS_Store' -type f -delete

# Files HU077022015.doc & HU029682017.docx are corrupt. Manually deleted from data/raw 
# (textract not dealing with Shell Error exceptions)

# Destination directory of txt files
dest_files_path = os.path.join(os.getcwd(), 'data/processed/txt_files')

# Loop to extract txt from word files (with decorator progress bar)
for word_file in  tqdm(os.listdir(docs_path)):

    file, extension = os.path.splitext(word_file)
    
    # Create txt file concatenating .txt extension to file name
    dest_file_name = file + '.txt'
    
    # Extract text from the file
    content = textract.process(os.path.join(docs_path, word_file))
    
    # Create and open new file & prepare to write the Binary Data (represented by wb - Write Binary)
    write_text_file = open(os.path.join(dest_files_path, dest_file_name), "wb")
    
    # Write the content and close the newly created file
    write_text_file.write(content)
    write_text_file.close()

100%|██████████| 35255/35255 [50:46<00:00, 11.57it/s]


Missing string for decision 48

In [22]:
with open('./data/processed/txt_files/DC000912019___DC001272019.txt', 'r') as file:
    decision_string = file.read()
# Open jsonData file as data
with open('./data/jsonDataFinal.json') as json_file:
    data = json.load(json_file)

# Obtain the full text of the court decision
data[48].update({'String': decision_string})

# Save data as a json file jsonDataFinal in data directory
with open('./data/jsonDataFinal.json', 'w') as fout:
    json.dump(data, fout)

# 3. Descriptive statistics on the downloaded files


This section provides some descriptive statistics on the downloaded files.

In [ ]:
def get_size(filename):
    st = os.stat(filename)
    return st.st_size

# Extract name and size of all files in docs_path
files_name_list_raw = [f for f in listdir(docs_path) if isfile(join(docs_path, f))]
files_size_list_raw = [get_size(join(docs_path, f)) for f in listdir(docs_path) if isfile(join(docs_path, f))]

# Obtain/check number of files
print(f'Number of files: {len(files_name_list_raw)}')

# Unique files based on size file_name
print(f'Number of unique file names: {len(set(files_name_list_raw))}')

# Unique files based on size file_name
print(f'Max file sizes: {max(files_size_list_raw)}')
print(f'Min of unique file sizes: {min(files_size_list_raw)}')

print(files_size_list_raw[0])


In [ ]:
# Loop over each text file and extract Court information
for text in os.listdir(txt_path):
    with open(txt_path + text, 'r') as file:
        files.append(text)
        #print(text)
        decision_string = file.read()
        # The strategy is to trim from both ends of the string

# Save data as a json file jsonDataFinal in data directory
with open('./data/jsonDataFinal.json', 'w') as fout:
    json.dump(data, fout)

print(f'Current working directory: {os.getcwd()}')

# Open jsonData file
jsonData_path = os.path.join(os.getcwd(), 'data/jsonData.json')
with open(jsonData_path) as json_file:
    data = json.load(json_file)
    print(json.dumps(data[32554], indent = 4, sort_keys = True))

#parsed = json.loads(jsonData)
#print(json.dumps(parsed[16366], indent = 4, sort_keys = True))


In [ ]:
#  Number of files

do sentence and token count (max min, average, etc.)


# Size of files
https://realpython.com/working-with-files-in-python/#getting-file-attributes

# doc vs docx
https://realpython.com/working-with-files-in-python/#filename-pattern-matching

# Number of dictionaries with sentence. Delete the rest?

# Longest sentence

# Shortest sentemce

# Number of reported vs unreported cases (use the name of the file to discriminate them)